# Find frequencies that make only $(\hat{a}^2\hat{b}^2\hat{e}^{\dagger2}+\hat{a}\hat{b}\hat{d}^2\hat{e}^{\dagger2})\hat{c}^\dagger +h.c.$ resonant in the 7th order expansion of $\sin(\hat{a}+\hat{b}+\hat{c}+\hat{d}+\hat{e}+h.c.)$

Proceed as in the 5th order example.

In [1]:
import hamnonlineng as hnle

In [2]:
letters = 'abcde'
resonant = [hnle.Monomial(1, 'aabbEEC'), hnle.Monomial(1,'abddEEC')]
op_sum = hnle.operator_sum(letters)
sine_exp = (
            hnle.sin_terms(op_sum, 3)
           +hnle.sin_terms(op_sum, 5)
           +hnle.sin_terms(op_sum, 7)
           )
off_resonant = hnle.drop_single_mode(
                 hnle.drop_definitely_offresonant(
                   hnle.drop_matching(sine_exp.m, resonant)))
off_resonant = list(off_resonant)
print('Number of off-resonant constraints: %d'%len(off_resonant))

Number of off-resonant constraints: 6308


Try to solve (takes around a minute):

In [3]:
res = hnle.head_and_count(
        hnle.solve_constraints_gecode(resonant, off_resonant, letters, maxfreq=50))

0 solutions.


Remove constraints on terms of the form $\hat{a}^2\hat{b}^2\dots$ or $\hat{a}\hat{b}\dots$ or those that do not contain any $\hat{a}$s or $\hat{b}$s.

In [4]:
# Drop all of the form aabb...
starts_with_aabb = [_ for _ in off_resonant if _.s[4:5].lower() not in 'ab' and _.s.startswith('AABB')]
# Drop all of the form ab...
starts_with_ab   = [_ for _ in off_resonant if _.s[2:3].lower() not in 'ab' and _.s.startswith('AB')]
# Drop all that do not contain any a or b
no_a_or_b = [_ for _ in off_resonant if 'a' not in _.s.lower() or 'b' not in _.s.lower()]
to_be_removed = starts_with_aabb + starts_with_ab + no_a_or_b
print('Number of constraints starting with ab or aabb or containing no a or b: %d'%len(to_be_removed))

Number of constraints starting with ab or aabb or containing no a or b: 3766


In [5]:
off_resonant = hnle.drop_matching(off_resonant, to_be_removed)
off_resonant = list(off_resonant)
print('Number of new off-resonant constraints: %d'%len(off_resonant))

Number of new off-resonant constraints: 2542


Try to solve again:

In [6]:
res = hnle.head_and_count(
        hnle.solve_constraints_gecode(resonant, off_resonant, letters, maxfreq=33))

[23, 25, 30, 24, 33]
[25, 23, 30, 24, 33]
2 solutions.


See which of the removed constraints fail:

For the first solution:

In [7]:
res[0]

[23, 25, 30, 24, 33]

In [8]:
hnle.filter_resonant(res[0], to_be_removed, letters)

[-1/12.AAAccDe, -1/36.AAACeee, -1/36.BBBDeee, -1/48.CddddEE]

For the second solution:

In [9]:
res[1]

[25, 23, 30, 24, 33]

In [10]:
hnle.filter_resonant(res[1], to_be_removed, letters)

[-1/36.AAADeee, -1/12.BBBccDe, -1/36.BBBCeee, -1/48.CddddEE]